In [36]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow import keras

import Rakuten_preprocessing as rkt
from importlib import reload 
reload(rkt)

from Rakuten_preprocessing import Rakuten_img_path

## Loading the data

In [32]:
folder_path = '../data/clean/'
data = pd.read_csv(os.path.join(folder_path, 'X_train.csv'), index_col=0)
target = pd.read_csv(os.path.join(folder_path, 'Y_train.csv'), index_col=0)
prdtype = pd.read_csv('../data/prdtype.csv', index_col='prdtypecode')
target = prdtype.loc[target['prdtypecode'], 'prdtypedesignation'].reset_index()

In [14]:
target

,prdtypecode,prdtypedesignation
0,10,Livres occasion
1,2280,Magazines occasion
2,50,Accessoires gaming
3,1280,Jouets enfants
4,2705,Livres neufs
...,...,...
84911,40,Jeux vidéo CDs équipement
84912,2583,Piscines et accessoires
84913,2280,Magazines occasion
84914,1560,Mobilier


In [44]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#Encoding classes as integers
encoder = LabelEncoder()
labels = encoder.fit_transform(target['prdtypedesignation'])

img_paths = Rakuten_img_path(img_folder=r"C:\Users\Julien Fournier\Documents\DST\RakutenProject\Data\images\image_train_resized",
                             imageid=data['imageid'], productid=data['productid'], suffix='_resized')

data_img = pd.DataFrame({'labels': labels, 'img_path': img_paths})

df_train, df_test = train_test_split(data_img, test_size=0.2, random_state=123)

In [45]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

#Data generator for the train and test sets
train_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                          rotation_range=20, width_shift_range=0.1, height_shift_range=0.1,
                                          zoom_range=0.1, horizontal_flip = True)

test_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_data_generator.flow_from_dataframe(dataframe=df_train, x_col='img_path', y_col='labels',
                                                           class_mode='raw', target_size=(224, 224),
                                                           batch_size=64)

test_generator = test_data_generator.flow_from_dataframe(dataframe=df_test, x_col='img_path', y_col='labels',
                                                           class_mode='raw', target_size=(224, 224),
                                                           batch_size=64)

Found 67932 validated image filenames.
Found 16984 validated image filenames.


In [52]:
from keras.layers import Input, Conv2D, Dense, MaxPooling2D, Dropout, Flatten
from keras.models import Sequential, Model
from keras import regularizers

num_class = len(np.unique(labels))

model = Sequential()
model.add(Conv2D(filters=64, input_shape=(224, 224, 3), kernel_size=(5,5), padding='valid', activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(MaxPooling2D(pool_size = (2, 2), strides=2, padding='valid'))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(units=num_class, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 220, 220, 64)      4864      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 110, 110, 64)      0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 110, 110, 64)      0         
                                                                 
 flatten_2 (Flatten)         (None, 774400)            0         
                                                                 
 dense_2 (Dense)             (None, 27)                20908827  
                                                                 
Total params: 20913691 (79.78 MB)
Trainable params: 20913691 (79.78 MB)
Non-trainable params: 0 (0.00 Byte)
____________

In [53]:
history = model.fit(x=train_generator, validation_data=test_generator, epochs=2, verbose=True)

Epoch 1/2


1062/1062 [==============================] - 1571s 1s/step - loss: 582.5247 - accuracy: 0.0943 - val_loss: 361.6511 - val_accuracy: 0.1256
Epoch 2/2
1062/1062 [==============================] - 951s 895ms/step - loss: 284.8131 - accuracy: 0.0991 - val_loss: 205.7016 - val_accuracy: 0.1395
